# Indiana Pines

1. Import dependencies

In [ ]:
import random
import torch
import multiprocessing
import numpy as np

import torch.utils.data as data

from sklearn.model_selection import train_test_split

from lightning import Trainer

from src.util.torch import resolve_torch_device
from src.util.hsi import (
    extract_patches,
    reduce_hsi_dim,
    train_test_band_patch_split,
    preprocess_hsi,
    PreProcessType,
    DimReductionType,
)
from src.data.indian_pines import load_indian_pines
from src.model.hsic import HyperSpectralImageClassifier
from src.visualization.plot import plot_segmentation_comparison
from src.data.dataset_decorator import UnlabeledDatasetDecorator
from src.visualization.plot import (
    plot_segmentation_comparison,
)
from src.util.reporting import (
    create_model_name,
    report_run,
    read_report_to_show,
    lightning_metrics,
)
from src.util.list_ext import smooth_moving_average
from src.visualization.plot import plot_epoch_generic, plot_epoch_generic_comparison
from src.model.fdssc import FDSSC_f

2. Prepare env

In [ ]:
learning_rate = 1e-4
weight_decay = 0
num_epochs = 30

In [ ]:
batch_size = 64
patch_size = 9
target_dim = 75

examples_per_class = []

pre_process_type = PreProcessType.STANDARTIZATION
dim_reduction_type = DimReductionType.PCA

In [ ]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.set_float32_matmul_precision("medium")

In [ ]:
f"Device is {device}"

3. Load dataset

In [ ]:
image, labels = load_indian_pines()

image_h, image_w, image_c = image.shape

In [ ]:
_, image = preprocess_hsi(image, pre_process_type)

In [ ]:
_, target_dim, image = reduce_hsi_dim(
    image, target_dim, dim_reduction_type, device, random_seed
)

In [ ]:
x, y = extract_patches(image, labels, patch_size=patch_size)

In [ ]:
num_classes = len(np.unique(y))

f"Number of classes {num_classes}"

In [ ]:
# examples_per_class = [20] * num_classes

In [ ]:
if examples_per_class:
    x_train, y_train, x_test, y_test, y_masked = train_test_band_patch_split(
        x, y, examples_per_class, "indian_pines"
    )

    plot_segmentation_comparison(
        y.reshape(image_h, image_w), y_masked.reshape(image_h, image_w)
    )
else:
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=random_seed, stratify=y
    )

# plot_numbers_distribution(y_train, desc="Train class distribution")

In [ ]:
x_tensor = torch.tensor(x, dtype=torch.float32).permute(0, 3, 1, 2) 
y_tensor = torch.tensor(y, dtype=torch.long)
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
cpu_count = multiprocessing.cpu_count()

f"Setting num_workers to {cpu_count}"

In [ ]:
train_dataset = data.TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = data.TensorDataset(x_test_tensor, y_test_tensor)
full_dataset = data.TensorDataset(x_tensor, y_tensor)

train_loader = data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=cpu_count,
    persistent_workers=True,
)
test_loader = data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=cpu_count,
    persistent_workers=True,
)
full_loader = data.DataLoader(
    full_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=cpu_count,
    persistent_workers=True,
)
predict_loader = data.DataLoader(
    UnlabeledDatasetDecorator(full_dataset),
    batch_size=batch_size,
    num_workers=cpu_count,
    persistent_workers=True,
)

In [ ]:
f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}"

4. Train model

In [ ]:
model = HyperSpectralImageClassifier(
    FDSSC_f(
        band=target_dim,
        classes=num_classes,
    ),
    num_classes,
    lr=learning_rate,
    weight_decay=weight_decay,
)

trainer = Trainer(accelerator="auto", max_epochs=num_epochs)

In [ ]:
trainer.fit(
    model,
    train_loader, 
    test_loader,
)

In [ ]:
smothed_train = smooth_moving_average([it.loss.cpu() for it in model.train_metrics], 20)
smothed_eval = smooth_moving_average([it.loss.cpu() for it in model.val_metrics], 20)

plot_epoch_generic_comparison(smothed_train, smothed_eval)

In [ ]:
smooth_f1 = smooth_moving_average([it.f1.cpu() for it in model.val_metrics], 20)

plot_epoch_generic(smooth_f1, desc="F1")

In [ ]:
validation_result = trainer.validate(model, full_loader)

validation_result

2. Display prediction

In [ ]:
y_pred = trainer.predict(model, predict_loader)

In [ ]:
y_pred = torch.cat(y_pred, dim=0)
y_pred = torch.argmax(y_pred, dim=1)
y_pred = y_pred.reshape(image_h, image_w)

In [ ]:
plot_segmentation_comparison(y.reshape(image_h, image_w), y_pred.numpy(), num_classes)

6. Write report

In [ ]:
model_name = create_model_name("indian_pines_", examples_per_class)
model_category = "fdssc"

report_run(
    model_name=model_name,
    model_category=model_category,
    run_desc="Default run",
    run_params={
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "num_epochs": num_epochs,
        "batch_size": batch_size,
        "patch_size": patch_size,
        "target_dim": target_dim,
        "pre_process_type": str(pre_process_type),
        "dim_reduction_type": str(dim_reduction_type),
    },
    run_metrics=lightning_metrics(validation_result),
)

In [ ]:
read_report_to_show(model_name, sort_by_metric="f1")

In [ ]:
read_report_to_show(model_name, sort_by_metric="f1", model_category=model_category)